#Motor AI Practical Exercise **1**

> This exercise is aimed at assessing skills about acquiring open source data as well as analyzing vector and raster data.



In [28]:
!pip install geemap

ERROR: Operation cancelled by user


In [31]:
!pip install pyrosm

ERROR: Operation cancelled by user


In [30]:
!pip install rasterio

ERROR: Operation cancelled by user


## Data Acquisition

> Obtain Sentinel 2 imagery for Berlin region using GEE



In [4]:
#Import necessary libraries
import ee
import geemap
import time
import requests
import geopandas as gpd
from pyrosm import OSM
import rasterio
from rasterio.mask import mask
import rasterio.warp
from rasterio.windows import Window

In [5]:
# Authenticate with Earth Engine
ee.Authenticate()

# Initialize the library
ee.Initialize(project='ee-silabas26')

In [6]:
# Define area of interest for Berlin region
berlin_aoi = ee.Geometry.Rectangle([13.0878, 52.3382, 13.7604, 52.6749])

# Load the Sentinel-2 Image Collection for the defined AOI and the date 2024-05-15
# in order to obtain images with least cloud coverage, cloud coverage percentage was defined as %1
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(berlin_aoi) \
    .filterDate('2024-05-14', '2024-05-16') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))

In [7]:
# Print the size of the S2 Image Collection
collection_size = sentinel2.size()
print("The size of Sentinel-2 Image Collection:", collection_size.getInfo())

The size of Sentinel-2 Image Collection: 5


In [8]:
# Inspect the attribute information of each image in the S2 Image Collection
image_list = sentinel2.toList(collection_size)
for i in range(collection_size.getInfo()):
    image = ee.Image(image_list.get(i))
    date = image.date().format("YYYY-MM-dd").getInfo()
    cloud_percentage = image.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()
    print(f"Image {i+1}: Date={date}, Cloudy Pixel Percentage={cloud_percentage}%")

Image 1: Date=2024-05-15, Cloudy Pixel Percentage=0.003563%
Image 2: Date=2024-05-15, Cloudy Pixel Percentage=0.015299%
Image 3: Date=2024-05-15, Cloudy Pixel Percentage=0.021961%
Image 4: Date=2024-05-15, Cloudy Pixel Percentage=0.007686%
Image 5: Date=2024-05-15, Cloudy Pixel Percentage=0.905308%


In [9]:
# Calculate the median image
# median(): reduces an image collection by calculating the median of all values at each pixel across the stack of all matching bands
median_image = sentinel2.median()

In [10]:
# Select the bands of interest (B4, B3, B2 for RGB)
bands = ['B4', 'B3', 'B2']
median_image = median_image.select(bands)

# Export S2 image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=median_image,
    description='Berlin_S2_RGB_20240515',
    folder='EarthEngineImages',
    fileNamePrefix='Berlin_S2_RGB_20240515',
    region=berlin_aoi,
    scale=10, #resolution in meters
    crs='EPSG:4326'
)
task.start()



In [ ]:
#Check task status
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)

print('Task status:', task.status())

Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id: H73AQTY6XDIDHXUMFFJ4HJZA).
Polling for task (id

In [11]:
#Create an interactive map with visualization parameters
visualization = {
    'min': 0,
    'max': 2000,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(13.4241, 52.50655, 10)
m.add_layer(sentinel2.mean(), visualization, 'RGB')
# Add Berlin AOI
m.addLayer(berlin_aoi, {
  'color': '00FF00', # Green
  'width': 2  # Outline thickness
}, 'Berlin Region')

m

Map(center=[52.50655, 13.4241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [12]:
#Download OSM data for Berlin from Geofabrik
url = "https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf"
response = requests.get(url)
with open("berlin-latest.osm.pbf", "wb") as file:
    file.write(response.content)

In [13]:
#Use 'pyrosm' library to extract the roads
osm = OSM("berlin-latest.osm.pbf")
roads = osm.get_network(network_type="driving")

## Main Processing Steps

In [14]:
# Filter roads by max speed and travel direction
filtered_roads = roads[(roads['maxspeed'].astype(str).str.extract('(\d+)')[0].astype(float) >= 50) &
                       (roads['oneway'] == 'yes')]

In [15]:
# Save buffered roads as GeoPackage
filtered_roads.to_file("/filtered_roads.gpkg", driver="GPKG")

In [20]:
# Convert data to GeoDataFrame
filtered_roads_gdf = gpd.GeoDataFrame(filtered_roads, geometry='geometry', crs='EPSG:4326')

# Reproject to EPSG:25833 before buffering
filtered_roads_gdf = filtered_roads_gdf.to_crs(epsg=25833)

In [21]:
# Buffer the roads by 20 meters
buffered_gdf = filtered_roads_gdf.buffer(20)

# Reproject back to EPSG:4326
buffered_gdf = buffered_gdf.to_crs(epsg=4326)

In [22]:
# Save buffered roads as GeoPackage
buffered_gdf.to_file("/buffered_roads.gpkg", driver="GPKG")

In [23]:
# Mount Google Drive in order to access Sentinel-2 raster
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
 #Clip the S2 imagery with respect the buffer and save
with rasterio.open("/content/drive/MyDrive/EarthEngineImages/Berlin_S2_RGB_20240515.tif") as src:
    out_image, out_transform = mask(src, buffered_gdf.geometry, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff", "height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform})

with rasterio.open("/Berlin_S2_clipped_image.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [26]:
#Reproject the clipped image to EPSG:25833 and save
with rasterio.open("/Berlin_S2_clipped_image.tif") as src:
    transform, width, height = rasterio.warp.calculate_default_transform(
        src.crs, 'EPSG:25833', src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': 'EPSG:25833',
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open('/Berlin_S2_reprojected_image_25833.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            rasterio.warp.reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs='EPSG:25833',
                resampling=rasterio.warp.Resampling.nearest)

In [27]:
 # Create a function to tile the reprojected image into 512 by 512 pixel image patches
def save_tile(dataset, window, filename):
    with rasterio.open(filename, 'w', driver='GTiff',
                       height=window.height, width=window.width,
                       count=dataset.count, dtype=dataset.dtypes[0],
                       crs=dataset.crs,
                       transform=dataset.window_transform(window),
                       compress='lzw') as dst:
        dst.write(dataset.read(window=window))

# Use save_tile function to tile the reprojected image and save image patches
with rasterio.open('/Berlin_S2_reprojected_image_25833.tif') as src:
    for i in range(0, src.width, 512):
        for j in range(0, src.height, 512):
            window = Window(i, j, 512, 512)
            save_tile(src, window, f'tile_{i}_{j}.jpegs')

